- Author: 
### Borja Delgado González
# - Objective: 
### Project for downloading video game information from the VGChartz website and storing the data in a .csv file or SQL database.


1. Import libraries to use in this project

In [29]:
# Libraries
import requests
import urllib
from bs4 import BeautifulSoup, element
import pandas as pd
import numpy as np
import sqlite3


2. Creation of the scraper class for later use


In [26]:
class VideoGameScraper:
    def __init__(self, url1, url2, pages):
        self.url1 = url1
        self.url2 = url2
        self.pages = pages
        self.session = requests.Session()

    def scrape(self):
        videogame = []
        platform = []
        editor = []
        developer = []
        sales_na = []
        sales_eu = []
        sales_jp = []
        sales_others = []
        sales_tot = []
        release_date = []
        genre = []

        for page in range(1, self.pages + 1):
            surl = self.url1 + str(page) + self.url2
            response = self.session.get(surl)
            soup = BeautifulSoup(response.text, "html.parser")

            videogame_tag = list(filter(lambda x: 'href' in x.attrs and x.attrs['href'].startswith('https://www.vgchartz.com/game/'),soup.find_all("a")))

            for tag in videogame_tag:
                videogame.append(tag.contents[0][:-4])
                data = tag.parent.parent.find_all("td")
                platform.append(data[3].find("img").attrs["alt"])
                editor.append(data[4].string)
                developer.append(data[5].string)
                sales_na.append(float(data[7].string[:-1]) if not data[7].string.startswith("N/A") else np.nan)
                sales_eu.append(float(data[8].string[:-1]) if not data[8].string.startswith("N/A") else np.nan)
                sales_jp.append(float(data[9].string[:-1]) if not data[9].string.startswith("N/A") else np.nan)
                sales_others.append(float(data[10].string[:-1]) if not data[10].string.startswith("N/A") else np.nan)
                sales_tot.append(float(data[6].string[:-1]) if not data[6].string.startswith("N/A") else np.nan)
                tag_date = data[11].string.split()[-1]
                release_year = np.int32("19" + tag_date) if int(tag_date) >= 80 else np.int32("20" + tag_date)
                release_date.append(release_year)
                #genre.append(tag.parent.parent.find_all("div", {"id": "gameGenInfoBox"})[0].find_next_sibling().string)
                genre_url = tag.attrs['href']
                genre_link = urllib.request.urlopen(genre_url).read()
                genre_soup = BeautifulSoup(genre_link, "html.parser")
      
                h2s = genre_soup.find("div", {"id": "gameGenInfoBox"}).find_all('h2')
        
                genre_tag = element.Tag
                for h2 in h2s:
                    if h2.string == 'Genre':
                        genre_tag = h2
                        #print(genre_tag)
                genre.append(genre_tag.next_sibling.string)

        df = pd.DataFrame({
            "videogame": videogame,
            "platform": platform,
            "editor": editor,
            "developer": developer,
            "sales_na": sales_na,
            "sales_eu": sales_eu,
            "sales_jp": sales_jp,
            "sales_otras": sales_others,
            "sales_tot": sales_tot,
            "release": release_date,
            "genre": genre
        })

        return df

if __name__ == "__main__":
    # Define url variables
    url1 = "https://www.vgchartz.com/games/games.php?page="
    url2 = '&results=1000&order=Sales&ownership=Both&direction=DESC&showtotalsales=1' # Here you can change the number after results to change the number of entries displayed by page.
    url2 += '&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1'
    url2 += '&showreleasedate=1&showlastupdate=0&showvgchartzscore=0&showcriticscore=0&showuserscore=0&showshipped=0'
    scraper = VideoGameScraper(url1, url2, pages=65) # 65 pages if 1000 entries are displayed in each page.

3. Create the scraper

In [27]:
scraper = VideoGameScraper(url1, url2, 65)

4. Launch it

In [ ]:
df = scraper.scrape()

5. Now we can decide wether we want to store the dataset in a .csv...

In [ ]:
df.to_csv('videogames_sales.csv')

6. Or to store it in a SQL database...

In [ ]:
# Connection to the database
conn = sqlite3.connect('videogames_sales_db.sqlite3')

cur = conn.cursor()

# Creation of a table to store the dataframe
cur.execute('''CREATE TABLE IF NOT EXISTS games (
    videogame TEXT,
    platform TEXT,
    editor TEXT,
    developer TEXT,
    sales_na FLOAT,
    sales_eu FLOAT,
    sales_jp FLOAT,
    sales_otras FLOAT,
    sales_tot FLOAT,
    release INTEGER,
    genre TEXT
)''')

# DataFrame into the table
df.to_sql('games', conn, if_exists='replace', index=False)

# Commit the changes
conn.commit()

# Close the connection
conn.close()